In [1]:
from selenium import webdriver
import urllib.request
import time

In [2]:
IS_LINUX = False

In [3]:
driver = webdriver.Firefox(executable_path="geckodriver.exe" if not IS_LINUX else "./chromedriver_linux")

##UA = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:25.0) Gecko/20100101 Firefox/25.0'
#PHANTOMJS_ARG = {'phantomjs.page.settings.userAgent': UA}

#driver = webdriver.PhantomJS(desired_capabilities=PHANTOMJS_ARG)

In [4]:
driver.get("https://tinder.com/app/recs")

### Connexion 

In [5]:
classbutton = "button Lts($ls-s) Z(0) CenterAlign Mx(a) Cur(p) Tt(u) Ell Px(24px) Px(20px)--s Py(0) Bdrs(0) Mih(40px) Fw($semibold) focus-button-style Fz($s) Bdrs(4px) Bg(#fff) C($c-pink) Bg($primary-gradient):h C(#fff):h"
python_button = driver.find_element_by_xpath('//*[@class="'+ classbutton +'"]')
python_button.click()

In [6]:
connect_phone_number = "Connexion avec un numéro de tél."
python_button = driver.find_element_by_xpath('//*[@aria-label="'+ connect_phone_number +'"]')
python_button.click()

In [7]:
driver.switch_to.default_content()
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))

In [8]:
python_button = driver.find_element_by_id("home_children_button")
python_button.click()

### TakeCaptcha
Ignorer cette cell

In [11]:
i = 11

img = driver.find_element_by_id("game_challengeItem_image")
src = img.get_attribute('src')
name = "image_0" + str(i) + ".png"
urllib.request.urlretrieve(src, name)
i += 1

filedata = urllib.request.urlopen(src)
datatowrite = filedata.read()
 
with open('/projet/ESGI/3ESGI/NoSQL/BotCapcha/' + name, 'wb') as f:
    f.write(datatowrite)

In [9]:
driver.switch_to.default_content()
phone_element = driver.find_element_by_name("phone_number")
phone_element.send_keys("0695776028")

In [69]:
def TakeInstagram():
    try:
        class_desc = "P(16px) Ta(start) Us(t) C($c-secondary) BreakWord Whs(pl) Fz($ms)"
        takedesc = driver.find_element_by_xpath('//*[@class="'+ class_desc +'"]')
        takedesc = takedesc.find_elements_by_css_selector("*")

        insta_list = [ "📷 ", "📸 ","@",
            "IG: ", "Ig: ","ig: ", "insta: ", "instagram: ", "Insta: ", "Instagram: ", 
            "IG:", "Ig:","ig:", "insta:", "instagram:", "Insta:", "Instagram:",
            "IG :", "Ig :","ig :", "insta :", "instagram :", "Insta :", "Instagram :",
            "IG : ", "Ig : ", "ig : ", "insta : ", "instagram : ", "Insta : ", "Instagram : ", 
            "IG ", "Ig ","ig ", "insta ", "instagram ", "Insta ", "Instagram "]

        description = takedesc[0].text
        instagram = ""
        description = description.split("\n")

        for desc_split in description:

            for insta_occurent in insta_list:

                result = desc_split.find(insta_occurent, 0, len(desc_split))
                if result != -1:
                    start = result + len(insta_occurent)
                    instagram = desc_split[start: len(desc_split)]
                    break

        instagram = instagram.split(" ")
        for word in instagram:
            
            if word != " ":
                instagram = word;
                
        return instagram
    
    except:
        
        return ""

In [52]:
def TakeTag():
    try:
        class_tag = "Mt(4px)"
        taketag = driver.find_element_by_xpath('//*[@class="'+ class_tag +'"]')
        taketag = taketag.find_elements_by_css_selector("*")
        
        list_tag = []
        
        for tag in taketag:
            list_tag.append(tag.text)
        
        
        return list_tag
    
    except:
        
        return []

In [53]:
def TakeName():
    try:
        class_name = "Fz($xl) Fw($bold) Fxs(1) Fxw(w) Pend(8px) M(0) D(i)"
        takename = driver.find_element_by_xpath('//*[@class="'+ class_name +'"]')
        
        return takename.text
    
    except:
        
        return ""

In [54]:
def TakeAge():
    try:
        class_age = "Whs(nw) Fz($l)"
        takeage = driver.find_element_by_xpath('//*[@class="'+ class_age +'"]')
        
        return takeage.text
    
    except:
        
        return ""

In [55]:
def TakePicture():
    try:
        class_picture = "profileCard__slider__img Z(-1)"
        takepicture = driver.find_element_by_xpath('//*[@class="'+ class_picture +'"]')
        style = takepicture.get_attribute("style")
        
        url = style[style.find("url") + 5: len(style)].split("\")")[0]
        
        return url
    
    except:
        
        return ""

In [71]:
#button_info_class = "Pos(a) D(f) Jc(sb) C(#fff) Ta(start) B(0) W(100%) Ai(fe) P(16px) P(20px)--l Cur(p) focus-button-style"
#python_info_button = driver.find_element_by_xpath('//*[@class="'+ button_info_class +'"]')
#python_info_button.click()
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

actions = ActionChains(driver)
actions.send_keys(Keys.ARROW_UP + 't')
actions.perform()
insta = TakeInstagram()

if insta != "":
    tag = TakeTag()
    name = TakeName()
    age = TakeAge()
    picture = TakePicture()
    
    print(insta)
    print(tag)
    print(name)
    print(age)
    print(picture)

alien.t
['Festivals', 'Escalade', 'Natation', 'Mode', 'Skateboarder']
Alienor
19
https://images-ssl.gotinder.com/5f07206dd4bafb0100f4c201/640x800_33bbe342-6c92-4b05-ac31-62f0eb87ff86.jpg


In [73]:
driver.close()